In [7]:
import numpy as np
import plotly.graph_objects as go

# 時間ステップとベクトル定義
num_frames = 40
t = np.linspace(0, 2 * np.pi, num_frames)

# 時間とともに回転するベクトル
vectors = np.array([
    [np.cos(theta), np.sin(theta), 0.5 * np.sin(2 * theta)]
    for theta in t
])

# フレーム作成（視点情報は含めない＝カメラ操作OK）
frames = []
for i in range(num_frames):
    u, v, w = vectors[i]
    frames.append(go.Frame(
        data=[
            go.Cone(
                x=[0], y=[0], z=[0],
                u=[u], v=[v], w=[w],
                sizemode="absolute",
                sizeref=0.5,
                anchor="tail",
                colorscale="Blues"
            )
        ],
        name=f"frame{i}"
    ))

# 初期ベクトル
u0, v0, w0 = vectors[0]

# スライダー定義
sliders = [dict(
    steps=[
        dict(method="animate",
             args=[[f"frame{k}"],
                   dict(mode="immediate",
                        frame=dict(duration=0, redraw=True),
                        transition=dict(duration=0))],
             label=f"{k}")
        for k in range(num_frames)
    ],
    transition=dict(duration=0),
    x=0.1,
    y=0,
    currentvalue=dict(font=dict(size=14), prefix="Frame: ", visible=True, xanchor="center"),
    len=0.9
)]

# Play/Pause ボタン
updatemenus = [
    dict(type="buttons",
         showactive=False,
         x=0.1,
         y=0.05,
         buttons=[
             dict(label="Play",
                  method="animate",
                  args=[None,
                        dict(frame=dict(duration=100, redraw=True),
                             fromcurrent=True,
                             transition=dict(duration=0))]),
             dict(label="Pause",
                  method="animate",
                  args=[[None], dict(frame=dict(duration=0, redraw=False),
                                     mode="immediate",
                                     transition=dict(duration=0))])
         ])
]

# 図の作成（初期カメラのみ設定＝再生中もマウス操作OK）
fig = go.Figure(
    data=[
        go.Cone(
            x=[0], y=[0], z=[0],
            u=[u0], v=[v0], w=[w0],
            sizemode="absolute",
            sizeref=0.5,
            anchor="tail",
            colorscale="Blues"
        )
    ],
    layout=go.Layout(
        scene=dict(
            xaxis=dict(range=[-2, 2]),
            yaxis=dict(range=[-2, 2]),
            zaxis=dict(range=[-2, 2]),
            camera=dict(eye=dict(x=0, y=0, z=1.5))  # 初期視点のみ設定
        ),
        updatemenus=updatemenus,
        sliders=sliders,
        margin=dict(l=0, r=0, t=0, b=0)
    ),
    frames=frames
)

fig.show()


In [5]:
from lib import *

In [36]:
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go
import numpy as np
from lib import *
from scipy.spatial.transform import Rotation as R

allData = load_all()

# Dash アプリの初期化
app = dash.Dash(__name__)

# ベクトルのアニメーション用データ生成
num_frames = 100
t = np.linspace(0, 2 * np.pi, num_frames)

# ベクトル（時間に伴って変化する）
vectors = np.array([
    [np.cos(theta), np.sin(theta), 0.5 * np.sin(2 * theta)]
    for theta in t
])

# 3D ベクトル表示用の関数
def create_figure(data):
    pos = data["pos"]; rot = data["rot"]
    xmin, ymin, zmin = np.min(pos, axis=0)
    xmax, ymax, zmax = np.max(pos, axis=0)
    base_vec = np.array([1, 0, 0])
    vec = [R.from_euler("xyz", r, degrees=True).apply(base_vec) for r in rot]

    fig = go.Figure(
        data=[
            go.Cone(
                x=[pos[0][0]], y=[pos[0][1]], z=[pos[0][2]],
                u=[vec[0][0]], v=[vec[0][1]], w=[vec[0][2]],
                sizemode="absolute",
                sizeref=0.5,
                anchor="tail",
                colorscale="Blues"
            )
        ],
        layout=go.Layout(
            scene=dict(
                xaxis=dict(range=[xmin, xmax]),
                yaxis=dict(range=[ymin, ymax]),
                zaxis=dict(range=[zmin, zmax]),
                camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))  # 初期視点
            ),
            updatemenus=[dict(
                type="buttons",
                x=0.1,
                y=0.05,
                buttons=[dict(
                    label="Play",
                    method="animate",
                    args=[None, {"frame": {"duration": 16, "redraw": True}, "fromcurrent": True}]
                )]
            ),
                dict(
                    type="buttons",
                    x=0.1,
                    y=0.15,
                    buttons=[dict(
                        label="Pause",
                        method="animate",
                        args=[[None], {"mode": "immediate", "frame": {"duration": 0, "redraw": False}}]
                    )]
                )
            ],
            sliders=[dict(
                steps=[dict(
                    method="animate",
                    args=[[f"{t}"], {"mode": "immediate", "frame": {"duration": 16, "redraw": True}}]
                ) for t in data["time"]],
                currentvalue=dict(visible=True, font=dict(size=14)),
                x=0.1,
                y=0
            )],
            margin=dict(l=0, r=0, t=0, b=0)
        ),
        frames=[go.Frame(
            data=[go.Cone(
                x=[p[0]], y=[p[1]], z=[p[2]],
                u=[v[0]], v=[v[1]], w=[v[2]],
                sizemode="absolute",
                sizeref=0.5,
                anchor="tail",
                colorscale="Blues"
            )],
            name=f"{t}"
        ) for t, p, v in zip(data["time"], pos, vec)]
    )
    return fig

# 初期ベクトル
initial_vector = allData[0][0][0]

# レイアウト
app.layout = html.Div([
    html.Div([
        "User-ID: ", dcc.Input(id="userId", type="number", value=3, step=1, min="3", max="6", style={'width': '50px'}),
        "UI-ID: ", dcc.Input(id="uiId", type="number", value=1, step=1, min="0", max="3", style={'width': '50px'}),
        "Task-ID: ", dcc.Input(id="taskId", type="number", value=1, step=1, min="0", max="8", style={'width': '50px'})
    ]),
    html.Div(id='error-message', style={'color': 'red'}),
    dcc.Graph(id='vector-graph', figure=create_figure(initial_vector))
])

# 入力されたベクトルに基づいて図を更新するコールバック
@app.callback(
    Output('vector-graph', 'figure'),
    Output('error-message', 'children'),
    Input("userId", "value"),
    Input("uiId", "value"),
    Input("taskId", "value")
)
def update_vector(userId, uiId, taskId):
    try:
        # ベクトル成分を入力から取得
        if userId is None or uiId is None or taskId is None:
            raise ValueError("すべてのフィールドに値を入力してください")
        userId = int(userId)-3; uiId = int(uiId); taskId = int(taskId)
        return create_figure(allData[userId][uiId][taskId]), ""
    except Exception as e:
        traceback.print_exc()
        return dash.no_update, f"⚠️ 入力エラー: {str(e)}"

# 実行
if __name__ == '__main__':
    app.run(debug=True)


In [23]:
data = allData[0][0][0]
data["pos"]

array([[23.513,  1.538, 24.065],
       [23.519,  1.538, 24.089],
       [23.525,  1.539, 24.12 ],
       ...,
       [23.2  ,  1.629, 24.051],
       [23.201,  1.629, 24.051],
       [23.201,  1.628, 24.051]], shape=(1848, 3))